In [ ]:
!pip install google-colab-selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
import multiprocessing
import tqdm
import numpy as np
import scipy.stats as st
import scipy
import numba
import math


from bokeh.io import export_svg
from google_colab_selenium import Chrome

#import biocircuits

# Plotting modules
import bokeh.io
import bokeh.plotting
from bokeh.models import LinearColorMapper, ColorBar
#from bokeh.io import export_svgs

bokeh.io.output_notebook()

# Line profiler (can install with conda install line_profiler)
# %load_ext line_profiler

# Analysis of molecular classifiers under shared resources

Molecular sequestration can realize a single node that process the sum weighted inputs and feed it into a smooth ReLU activation function. Sigma factors are a great candidates to implement the biomolecular neural networks. However, multiple sigma factors will compete for the core DNApol. Therefore, how will this interaction affect the deicision boundaries?

To answer this question, we will develop a mathematical model that incorporate the competition for resoures, and quantify the effect of compeition. Moreover, to identify simple design rules for molecular classifiers.

Next, we will model multi layer neural networks and characterize the input-output behavior of molecular classsifiers.

Our hypothesis is that compeition for resouces can add a bias. This bias could depend as a fucntion of the concentration of the competitive species in a non-linear way. We will identify under what condition, we can still predict the linear classifiers and compose multi-layer neural networks.


## Chemical Reactions

\begin{align}
\emptyset \xrightarrow[]{\alpha_1} S_1 && S_1  \xrightarrow{\delta} \emptyset \\
\emptyset \xrightarrow[]{\beta_1} A_1 && A_1  \xrightarrow{\delta} \emptyset \\
S_1 + A_1 \xrightarrow{\gamma_1} \emptyset &&  S_1 + C \xrightarrow{\gamma_2} C_1\\
\emptyset \xrightarrow[]{\alpha_2} S_2 && S_2  \xrightarrow{\delta} \emptyset \\
\emptyset \xrightarrow[]{\beta_2} A_2 && A_2  \xrightarrow{\delta} \emptyset \\
S_2 + A_2 \xrightarrow{\gamma_1} \emptyset &&  S_2 + C \xrightarrow{\gamma_2} C_2
\end{align}

## Mathematical Modeling

\begin{align}
  \dot s_1 &= \alpha_1 - \delta s_1 - \gamma_1 s_1a_1 - \gamma_2 s_1c\\
  \dot a_1 &= \beta_1 - \delta a_1 - \gamma_1 s_1a_1 \\
  \dot s_2 &= \alpha_2 - \delta s_2 - \gamma_1 s_2a_2 - \gamma_2 s_2c\\
  \dot a_2 &= \beta_2 - \delta a_2 - \gamma_1 s_2a_2 \\
  \dot c_1 &= \gamma_2 s_1c - \delta c_1\\
  \dot c_2 &= \gamma_2 s_2c - \delta c_2
\end{align}
with mass conservation $c + c_1 + c_2 = c^{tot}$.
## Steady state analysis

\begin{align}
  \bar a_1 &= \frac{\alpha_1 - \delta \bar s_1 - \gamma_2 \bar s_1 \bar c}{\gamma_1 \bar  s_1} = \frac{\beta_1}{\gamma_1 \bar s_1 + \delta} \\
  \bar a_2 &= \frac{\alpha_2 - \delta \bar s_2 - \gamma_2 \bar s_2 \bar c}{\gamma_1 \bar  s_2} = \frac{\beta_2}{\gamma_1 \bar s_2 + \delta} \\
  \bar c_1 &= \frac{\gamma_2}{\delta} \bar s_1 \bar c \\
  \bar c_2 &= \frac{\gamma_2}{\delta} \bar s_2 \bar c \\
\end{align}

Then, we can use the mass conservation of $c$ and find that
\begin{align}
  \bar c = \frac{1}{1 + \frac{\gamma_2}{\delta} s_1 + \frac{\gamma_2}{\delta} s_2}c^{tot}
\end{align}

Then, we can find that
\begin{align}
  \bar c_1 &= \frac{s_1}{\frac{\delta}{\gamma_2} +  s_1 + s_2}c^{tot} \\
  \bar c_2 &= \frac{s_2}{\frac{\delta}{\gamma_2} +  s_1 + s_2}c^{tot} \\
\end{align}

Now, we are ready to find the null-clines of the system by finding $\bar c$
\begin{align}
  \gamma_2 \bar s_1 \bar c  &= \delta \bar c_1 = \alpha_1 - \delta \bar s_1 - \frac{\bar s_1}{\bar s_1 + \frac{\delta}{\gamma_1}}\beta_1 \\
  \gamma_2 \bar s_2 \bar c  &= \delta \bar c_2 = \alpha_2 - \delta \bar s_2 - \frac{\bar s_2}{\bar s_2 + \frac{\delta}{\gamma_1}}\beta_2
\end{align}

\begin{align}
  \frac{\bar c_1}{\bar c_2}  &= \frac{\bar s_1}{\bar s_2} =  \frac{\alpha_1 - \delta \bar s_1 - \frac{\bar s_1}{\bar s_1 + \frac{\delta}{\gamma_1}}\beta_1}{\alpha_2 - \delta \bar s_2 - \frac{\bar s_2}{\bar s_2 + \frac{\delta}{\gamma_1}}\beta_2}
\end{align}

\begin{align}
  \alpha_2 \bar s_1  - \frac{\bar s_1 \bar s_2}{\bar s_2 + \delta/\gamma_1}\beta_2 = \alpha_1 \bar s_2 - \frac{\bar s_1 \bar s_2}{\bar s_1 + \delta/\gamma_1}\beta_1 \\
  \frac{\bar s_1}{\bar s_2} = \frac{\alpha_1 - \frac{\bar s_1 }{\bar s_1 + \delta/\gamma_1}\beta_1}{\alpha_2  - \frac{ \bar s_2}{\bar s_2 + \delta/\gamma_1}\beta_2}
\end{align}
\begin{align}
  \frac{\bar s_1}{\alpha_1 - \frac{\bar s_1 }{\bar s_1 + \delta/\gamma_1}\beta_1} = \frac{\bar s_2}{\alpha_2  - \frac{ \bar s_2}{\bar s_2 + \delta/\gamma_1}\beta_2} = z
\end{align}
\begin{align}
 s_i (s_i + \delta/\gamma_1) & = z(\alpha_i(s_i + \delta/\gamma_1) - \beta_i s_i) \\
 s_i^2 + (\delta/\gamma_1 - z(\alpha_i-\beta_i)) s_i - z\delta/\gamma_1 &= 0
\end{align}

This leads to a single positive real solution
\begin{align}
 s_i = \frac{1}{2} \left ( z(\alpha_i-\beta_i) - \delta/\gamma_1 + \sqrt{ (z(\alpha_i-\beta_i) - \delta/\gamma_1)^2 + 4z\delta/\gamma_1}\right)
\end{align}
In the fast sequestration regime, we can approximate it as
\begin{align}
 s_i \approx \max \left (0,  z(\alpha_i - \beta_i)\right )
\end{align}

\begin{align}
  \alpha_1 - \delta \bar s_1 -  <<\delta>>\frac{ \bar s_1}{\frac{\delta}{\gamma_2} + s_1 +  s_2}c^{tot} = \frac{\gamma_1 \bar  s_1}{\gamma_1 \bar s_1 + \delta} \beta_1
\end{align}

We assume that binding is faster than the dilution rate. Then, we get
\begin{align}
   \frac{ \bar s_1}{\frac{\delta}{\gamma_2} + s_1 +  s_2} \approx  \frac{ \bar s_1}{s_1 +  s_2} \approx \frac{ \max(0,\alpha_1-\beta_1)}{\max (0, \alpha_1-\beta_1) + \max (0, \alpha_2 -\beta_2)} = \xi
\end{align}

Then, we can rewrite the above term for sequestration as

\begin{align}
  \alpha_1 -  \xi c^{tot} - \delta \bar s_1  = \frac{\gamma_1 \bar  s_1}{\gamma_1 \bar s_1 + \delta} \beta_1
\end{align}
We can approximate the solution of the above equation as
\begin{align}
\bar s_1 = \max \left (0, \frac{\alpha_1 - \beta_1 -\xi c^{tot}}{\delta} \right) \\
\bar s_2 = \max \left (0, \frac{\alpha_2 - \beta_2 -(1-\xi) c^{tot}}{\delta} \right) \\
\end{align}

In [ ]:
def Sequestration_rhs(x,t,a1,b1,a2,b2,g1,g2,d,ct):
    S1, A1, S2, A2, C1, C2 = x
    C = ct - C1 - C2
    return np.array(
        [
            a1 - d*S1 - g1*A1*S1 - g2*S1*C,
            b1 - d*A1 - g1*A1*S1,
            a2 - d*S2 - g1*A2*S2 - g2*S2*C,
            b2 - d*A2 - g1*A2*S2,
            g2*S1*C - d*C1,
            g2*S2*C - d*C2,
        ]
    )

x0 = np.array([0., 0., 0., 0., 0., 0.])
a1 = 1
b1 = 0.5
a2 = 0
b2 = 0

g1 = 100
g2 = 100

d = 1
ct = 1

t = np.linspace(0,15,51)
x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(a1, b1, a2, b2,  g1, g2, d, ct))
x = x.transpose()

# Set up plots
fig_size = [150, 115] # for paper size (144, 115) # for larger size (144,144)
x_range = (0, 6)
y_range = (0, 0.61)

plots = []
for i in range(1):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"


plots[0].line(t,x[0,:], line_width=2, color="black")
plots[0].line(t,x[1,:], line_width=2, color="orange")

plots[0].circle(t[10:41:10],x[0,10:41:10], line_width=2, color="black")


bokeh.io.show(bokeh.layouts.row(plots))

## How does the strength of sequestration affect the decision boundary

In [ ]:
g1 = [10,100,1000] # sequestration rate
g2 = 10 # active complex rate
d = 1
ct = 1/5
a2 = 0
b2 = 0


tN = 100
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, g1k in enumerate(g1):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2, b2,  g1k, g2, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(g1):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1





Transient maps

In [ ]:
g1 = [100,1000]
output1 = np.zeros((N,N,3,100))
output2 = np.zeros((N,N,3,100))
transient_plots = np.zeros((N,N,2,100))
for k, g1k in enumerate(g1):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2, b2,  g1k, g2, d, ct))
            output1[j,i,k,:] = x.transpose()[0,:] # 0 (s1) 4 (c1)
            output2[j,i,k,:] = x.transpose()[4,:]/ct # 0 (s1) 4 (c1)
            transient_plots[j,i,k,:] = x.transpose()[4,:]/ct


# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(12):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k in range(6):
    z0 = np.c_[output2[:,:,0,5*k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,1,5*k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+6].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

webdriver = Chrome()

fig = bokeh.layouts.row(plots[0:6])
fig2 = bokeh.layouts.row(plots[6:12])
bokeh.io.show(bokeh.layouts.row(plots[0:6])) # Top r=0.1
bokeh.io.show(bokeh.layouts.row(plots[6:12])) # botton r=0.01
export_svg(fig, filename="fig3c_transient_0_5_10_15_20_25_r_01.svg",webdriver=webdriver)
export_svg(fig2, filename="fig3c_transient_0_5_10_15_20_25_r_001.svg",webdriver=webdriver)


plots2 = []
for i in range(2):
    plots2.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=(t[0],t[-1]), y_range=(-0.1,1)),)
plots2[0].line(t,transient_plots[0,-1,0,:],color='blue')
plots2[0].line(t,transient_plots[-1,-1,0,:],color='orange')
plots2[0].line(t,transient_plots[0,0,0,:],color='brown')
plots2[0].line(t,transient_plots[-1,0,0,:],color='black')
plots2[1].line(t,transient_plots[0,-1,1,:],color='blue')
plots2[1].line(t,transient_plots[-1,-1,1,:],color='orange')
plots2[1].line(t,transient_plots[0,0,1,:],color='brown')
plots2[1].line(t,transient_plots[-1,0,1,:],color='black')


bokeh.io.show(bokeh.layouts.row(plots2[0:2]))
fig3 = bokeh.layouts.row(plots2[0:2])
export_svg(fig3, filename="fig3c_transient_time_plots.svg",webdriver=webdriver)


<IPython.core.display.Javascript object>

['fig3c_transient_time_plots.svg']

## Tuning the effect of sigma factor 2 in the decision boundary

In [ ]:
g1 = 100.
g2 = 10
d = 1
ct = 1/5
a2 = [0, 0.5, 1]
b2 = 0

tN = 100
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))



for k, a2k in enumerate(a2):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2k, b2,  g1, g2, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(a2):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

Transient response

In [ ]:
g1 = 100.
g2 = 10
d = 1
ct = 1/5
a2 = [0.5, 1]
b2 = 0

tN = 100
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,2,100))
output2 = np.zeros((N,N,2,100))


for k, a2k in enumerate(a2):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2k, b2,  g1, g2, d, ct))
            output1[j,i,k,:] = x.transpose()[0,:] # 0 (s1) 4 (c1)
            output2[j,i,k,:] = x.transpose()[4,:]/ct # 0 (s1) 4 (c1)


# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(12):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k in range(6):
    z0 = np.c_[output2[:,:,0,5*k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,1,5*k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+6].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

webdriver = Chrome()

fig = bokeh.layouts.row(plots[0:6])
fig2 = bokeh.layouts.row(plots[6:12])
bokeh.io.show(bokeh.layouts.row(plots[0:6])) # Top r=0.1
bokeh.io.show(bokeh.layouts.row(plots[6:12])) # botton r=0.01
export_svg(fig, filename="fig3e_transient_0_5_10_15_20_25_alpha_05.svg",webdriver=webdriver)
export_svg(fig2, filename="fig3e_transient_0_5_10_15_20_25_alpha_1.svg",webdriver=webdriver)


plots2 = []
for i in range(2):
    plots2.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=(t[0],t[-1]), y_range=(-0.1,1)),)
plots2[0].line(t,output2[0,-1,0,:],color='blue')
plots2[0].line(t,output2[-1,-1,0,:],color='orange')
plots2[0].line(t,output2[0,0,0,:],color='brown')
plots2[0].line(t,output2[-1,0,0,:],color='black')
plots2[1].line(t,output2[0,-1,1,:],color='blue')
plots2[1].line(t,output2[-1,-1,1,:],color='orange')
plots2[1].line(t,output2[0,0,1,:],color='brown')
plots2[1].line(t,output2[-1,0,1,:],color='black')


bokeh.io.show(bokeh.layouts.row(plots2[0:2]))
fig3 = bokeh.layouts.row(plots2[0:2])
export_svg(fig3, filename="fig3e_transient_time_plots_alphas.svg",webdriver=webdriver)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

['fig3e_transient_time_plots_alphas.svg']

## The effect of anti-sigma factor 2

In [ ]:
g1 = 100.
g2 = 10
d = 1
ct = 1/5
a2 = 0.5
b2 = [0, 0.5, 1]


N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, b2k in enumerate(b2):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2, b2k,  g1, g2, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(b2):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

Transient response

In [ ]:
g1 = 100.
g2 = 10
d = 1
ct = 1/5
a2 = 0.5
b2 = [0.5, 1]

tN = 100
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,2,100))
output2 = np.zeros((N,N,2,100))

for k, b2k in enumerate(b2):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2, b2k,  g1, g2, d, ct))
            output1[j,i,k,:] = x.transpose()[0,:] # 0 (s1) 4 (c1)
            output2[j,i,k,:] = x.transpose()[4,:]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(12):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k in range(6):
    z0 = np.c_[output2[:,:,0,5*k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,1,5*k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+6].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

webdriver = Chrome()

fig = bokeh.layouts.row(plots[0:6])
fig2 = bokeh.layouts.row(plots[6:12])
bokeh.io.show(bokeh.layouts.row(plots[0:6])) # Top r=0.1
bokeh.io.show(bokeh.layouts.row(plots[6:12])) # botton r=0.01
export_svg(fig, filename="fig3f_transient_0_5_10_15_20_25_beta_05.svg",webdriver=webdriver)
export_svg(fig2, filename="fig3f_transient_0_5_10_15_20_25_beta_1.svg",webdriver=webdriver)


plots2 = []
for i in range(2):
    plots2.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=(t[0],t[-1]), y_range=(-0.1,1)),)
plots2[0].line(t,output2[0,-1,0,:],color='blue')
plots2[0].line(t,output2[-1,-1,0,:],color='orange')
plots2[0].line(t,output2[0,0,0,:],color='brown')
plots2[0].line(t,output2[-1,0,0,:],color='black')
plots2[1].line(t,output2[0,-1,1,:],color='blue')
plots2[1].line(t,output2[-1,-1,1,:],color='orange')
plots2[1].line(t,output2[0,0,1,:],color='brown')
plots2[1].line(t,output2[-1,0,1,:],color='black')


bokeh.io.show(bokeh.layouts.row(plots2[0:2]))
fig3 = bokeh.layouts.row(plots2[0:2])
export_svg(fig3, filename="fig3f_transient_time_plots_betas.svg",webdriver=webdriver)

<IPython.core.display.Javascript object>

['fig3f_transient_time_plots_betas.svg']

## Effect of sequestration rate 2 on the decision boundary

In [ ]:
g1 = 100 #[10,100,1000]
g2 = [1,10,100]
d = 1
ct = 1/5
a2 = 0
b2 = 0



N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, g2k in enumerate(g2):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2, b2,  g1, g2k, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(g2):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

## Weight 1 tuning decision boundary

In [ ]:
g1 = 100
g2 = 10
d = 1
ct = 1/5
a2 = 0
b2 = 0

w1 = [0.5, 1, 2]

N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, w1k in enumerate(w1):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(w1k*x1i, x2j, a2, b2,  g1, g2, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(w1):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

## Weight 1 tuning in shared resources

In [ ]:
g1 = 100
g2 = 10
d = 1
ct = 1/5
a2 = 0.5
b2 = 0

w1 = [0.5, 1, 2]

N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, w1k in enumerate(w1):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(w1k*x1i, x2j, a2, b2,  g1, g2, d, ct))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ct # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(w1):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1

## Effects of cT in the linear decision boundary

In [ ]:
g1 = 100
g2 = 10
d = 1
ct = [0.2, 0.5, 0.8]
a2 = 0
b2 = 0

#w1 = [0.5, 1, 2]

N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)

x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

for k, ctk in enumerate(ct):
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(x1i, x2j, a2, b2,  g1, g2, d, ctk))
            output1[j,i,k] = x.transpose()[0,-1] # 0 (s1) 4 (c1)
            output2[j,i,k] = x.transpose()[4,-1]/ctk # 0 (s1) 4 (c1)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(ct):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top S1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # botton C1